#BLIP (Boostrapping Langauge-Image Pretraining)
BLIP represents a significant advancement in the intersection of natural language processing (NLP) and computer vision. BLIP, designed to improve AI models, enhances their ability to understand and generate image descriptions. It learns to associate images with relevant text, allowing it to generate captions, answer image-related questions, and support image-based search queries.

In [5]:
!pip install transformers

In [6]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

Base model

In [7]:
#INitialize the processor and model from hugging face
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [8]:
#loading the image
image = Image.open("/content/IMG_0077.JPG")
display(image)

FileNotFoundError: [Errno 2] No such file or directory: '/content/IMG_0077.JPG'

In [ ]:
#Prepare the image
inputs = processor(image,return_tensors="pt")

In [ ]:
#Genrate captions
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0],skip_specail_tokens=True)
print("Generated Captions: ", caption)

large model

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model= BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [ ]:
image = image.convert('RGB')

In [ ]:
#conditional captioning
text ="a photo of "
inputs = processor(image, text, return_tensors="pt")


In [ ]:
out =model.generate(**inputs)
print(processor.decode(out[0],skip_specail_tokens=True))


In [ ]:
#unconditional captioning
inputs= processor(image,return_tensors="pt")
out= model.generate(**inputs)
print(processor.decode(out[0],skip_specail_tokens=True))


#Visual Question Answering

Salesforce/blip-vqa-base

In [9]:
from transformers import BlipForQuestionAnswering

In [10]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model =BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
# Define the question for VQA
question = "How many butterfly in the picture?"

# Prepare the inputs for the model
inputs = processor(image, question, return_tensors="pt")

# Generate the answer using the model
outputs = model.generate(**inputs)
answer = processor.decode(outputs[0], skip_special_tokens=True)

print("Generated Answer:", answer)

# Creating the user Interface using Gardio

In [ ]:
!pip install gradio

In [11]:
import gradio as gr

In [12]:
#For Caption Generation
processor_caption = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model_caption= BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

#For Question and Answer
processor_qanda = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_qanda = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

The input and output arguments take one or more Gradio components. As we'll see, Gradio includes more than 30 built-in components (such as the gr.Textbox(), gr.Image(), and gr.HTML() components) that are designed for machine learning applications

In [18]:
#for image caption
def generate_caption(image):
  inputs=processor_caption(images=image, return_tensors="pt")
  outputs=model_caption.generate(**inputs)
  caption=processor_caption.decode(outputs[0],skip_specail_tokens=True)
  return caption

#PIL Image input and returns a caption.
def caption_image(image):
  try:
    caption = generate_caption(image)
    return caption
  except:
    print("Error")

def answer_generation(image,question):
  try:
    ans = question_and_answer(image,question)
    return ans
  except:
    print("Error")

In [15]:
iface = gr.Interface(
    fn=caption_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Captioning with BLIP",
    description="Upload an image to generate a caption."
    )
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0c929b631df854b612.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
